In [1]:
#!pip install pyarrow
import numpy as np
import pandas as pd
import os

print(os.getcwd())

os.chdir('../../')
print(os.getcwd())
os.chdir('./Data')
os.getcwd()




c:\Users\chris.slim\OneWorkplace\TRACK Australia - General\Clients & Projects\Volkswagen Group Australia\Volkswagen\Data and Tech\ID Series Brief\4. POC Update\Analysis\Python
c:\Users\chris.slim\OneWorkplace\TRACK Australia - General\Clients & Projects\Volkswagen Group Australia\Volkswagen\Data and Tech\ID Series Brief\4. POC Update


'c:\\Users\\chris.slim\\OneWorkplace\\TRACK Australia - General\\Clients & Projects\\Volkswagen Group Australia\\Volkswagen\\Data and Tech\\ID Series Brief\\4. POC Update\\Data'

In [2]:
# Clean AAA data (2020-2022)

# !pip install openpyxl


df = pd.read_excel('1_inputs/ev_vehicles/vehicle_index_jan_2024.xlsx', sheet_name='Registration Numbers', dtype={'Postcode':str} )
df['Postcode'] = df['Postcode'].str.zfill(4)
print(df.shape[0])
print(df.dtypes)
df.rename(columns={'Registrations as at 31 January 2021':'2020'
                   , 'Registrations as at 31 January 2022':'2021'
                   , 'Registrations as at 31 January 2023':'2022'}
                   , inplace = True)


df = df.melt(id_vars = ['Postcode', 'State', 'Fuel Type'], value_vars=['2020','2021','2022'], var_name = 'Year', value_name = 'LVs')

print(df.shape[0])
print(df.head())
os.getcwd()



10548
Postcode                               object
State                                  object
Fuel Type                              object
Registrations as at 31 January 2021     int64
Registrations as at 31 January 2022     int64
Registrations as at 31 January 2023     int64
dtype: object
31644
  Postcode State    Fuel Type  Year    LVs
0     0800    NT          ICE  2020   6780
1     0800    NT          BEV  2020      3
2     0800    NT        HFCEV  2020      0
3     0800    NT  Hybrid/PHEV  2020    131
4     0810    NT          ICE  2020  21870


'c:\\Users\\chris.slim\\OneWorkplace\\TRACK Australia - General\\Clients & Projects\\Volkswagen Group Australia\\Volkswagen\\Data and Tech\\ID Series Brief\\4. POC Update\\Data'

In [3]:
# Clean EV Council Data (Tesla and BYD only, 2023)

ev_2023 = pd.read_excel('1_inputs/ev_vehicles/ev_council_sales_2023_original.xlsx', sheet_name = 'EV Council (Via ABC) - data-OfR')


# Aggregate to account for duplicated rows across suburbs
ev_2023 = ev_2023.groupby(['Postcode'], as_index=False).agg({'Total Orders': 'sum'})


# populate additional data fields, and some field cleaning
ev_2023['Year'] = 2023
ev_2023['Fuel Type'] = 'BEV'
ev_2023.rename(columns={'Total Orders' : 'LVs'}, inplace=True)
ev_2023['Postcode'] = ev_2023['Postcode'].astype('string').str.zfill(4)


# Infer state from postcode from original df
# print(df.groupby(['Postcode'], as_index = False)['State'].nunique().sort_values(by='State',ascending=False)) # Based on this, each post code has a unique state

postcode_lkp = df.copy().groupby(['Postcode', 'State'], as_index = False)['Year'].max().drop('Year', axis=1)
ev_2023 = pd.merge(left = ev_2023, right = postcode_lkp, left_on = 'Postcode', right_on = 'Postcode', how='left')

# Create 2023 all vehicles data
other_vehicles_2023 = df[(df['Year'] == '2022') & ~(df['Fuel Type'] == 'BEV')].copy()
other_vehicles_2023['Year'] = 2023
df = pd.concat([df, other_vehicles_2023, ev_2023], axis = 0)


# Save vehicle index to staging folder
while True:
    df.to_csv('2_staging/vehicle_index_clean.csv')
    print('Dataframe written to CSV successfully')
    break



Dataframe written to CSV successfully


In [4]:
# LOAD AND CLEAN EV RYIs
# ONLY RUN IF YOU HAVE NEW RYI DATA

ryi_customers = pd.read_csv('1_inputs/ryis/id_ryi_raw_data_update.csv', dtype={'Zip/Postal Code':str}, encoding='iso-8859-1')
print('before clean: {}'.format(ryi_customers.shape[0]))
ryi_customers.rename(columns={'Zip/Postal Code': 'Postcode'}, inplace=True)
ryi_customers['Postcode'] = ryi_customers['Postcode'].str.zfill(4)
print(ryi_customers.dtypes)

first_name_check = (ryi_customers['First Name'].str.lower().str.contains('test')) 

last_name_list = ['challen', 'cvet', 'test']
last_name_list = '|'.join(last_name_list)
last_name_check = ryi_customers['Last Name'].str.lower().str.contains(last_name_list)

email_list = ['%@%volkswagen','katzion','%test%@','cvet']
email_list = '|'.join(email_list)
email_check = ryi_customers['Email'].str.lower().str.contains(email_list)


ryi_customers = ryi_customers[~first_name_check & ~email_check & ~last_name_check]

print('after clean: {}'.format(ryi_customers.shape[0]))

ryi_customers = ryi_customers[['First Name', 'Last Name', 'Email', 'Postcode']].drop_duplicates()
print(ryi_customers.head())
print('after deduplication: {}'.format(ryi_customers.shape[0]))

while True:
    ryi_customers.to_csv('2_staging/ryi_clean_list_check.csv')
    print('Raw RYI data written to CSV')
    break


ryi = ryi_customers.groupby('Postcode', as_index=False)['Email'].count().rename(columns={'Email':'RYIs'})

ryi.sort_values( 'RYIs',ascending=False)

while True:
        ryi.to_csv('2_staging/id_ryi_clean.csv')
        print('Postcode level RYIs written to CSV')
        break



before clean: 6689
Unnamed: 0                 int64
First Name                object
Last Name                 object
Title                    float64
Company / Account        float64
Email                     object
Lead Source               object
Street                   float64
Rating                   float64
Postcode                  object
Model of Interest         object
2nd Model of Interest     object
dtype: object
after clean: 6651
  First Name  Last Name                        Email Postcode
0      Henry     Larsen  05_graphs.juries@icloud.com     2093
1  Sebastian       Ross   06settles_dicta@icloud.com     4217
2       Brad   Saunders             080978@gmail.com     2118
3        Sam  Denniston  09-fracas-pylons@icloud.com     3196
4    Timothy    Dowsett    0-gilts-advice@icloud.com     2073
after deduplication: 5668
Raw RYI data written to CSV
Postcode level RYIs written to CSV


In [5]:
# LOAD ALL OWNERS
# ONLY RUN IF YOU HAVE NEW OWNERSHIPS DATA

vwo = pd.read_csv('1_inputs/vw_owners/vw_owners_raw_data_update.csv'
                  , encoding='utf-8'
                  , parse_dates = ['Purchase Date', 'Created Date']
                  , dayfirst=True
                  , dtype={'Customer Postcode':str, 'Account: Zip/Postal Code':str})


In [12]:
# CLEAN OWNERSHIPS
# ONLY RUN IF YOU HAVE NEW OWNERSHIPS DATA


# DROP DUPLICATES

print('Before cleaning: {}'.format(vwo.shape[0]))

vwo = vwo.drop_duplicates(subset = ['VIN', 'Owner Name: Account Name', 'Purchase Date', 'Customer Postcode'])

print('After dedupe: {}'.format(vwo.shape[0]))

last_owner = vwo.sort_values('Purchase Date').groupby('VIN').tail(1)

print('Latest owner only: {}'.format(last_owner.shape[0]))


# ROLL UP TO POSTCODE LEVEL

owners = last_owner.groupby(['Customer Postcode'], as_index=False).agg(VWs = ('VIN', pd.Series.count))
owners.rename(columns={'Customer Postcode':'Postcode'}, inplace = True)
owners['Postcode'] = owners['Postcode'].str.zfill(4)

owners = owners.groupby('Postcode', as_index=False)['VWs'].sum()

while True:
        owners.to_csv('2_staging/vw_owners_clean.csv')
        print('Postcode level Ownerships written to CSV')
        break

Before cleaning: 647839
After dedupe: 647839
Latest owner only: 581910
Postcode level Ownerships written to CSV


In [7]:
## Useful code:
## Check VIN counts > 1 in last_owner 
#multiples = last_owner[last_owner.groupby('VIN')['Ownership Name'].transform('size') > 1]


# print(last_owner['Customer Postcode'].count())
# print(last_owner['Account: Zip/Postal Code'].count())
# non_matching = (vwo['Customer Postcode'] != vwo['Account: Zip/Postal Code'])
# print(vwo[non_matching])

In [17]:
df.head()

,Postcode,State,Fuel Type,Year,LVs
0,0800,NT,ICE,2020,6780
1,0800,NT,BEV,2020,3
2,0800,NT,HFCEV,2020,0
3,0800,NT,Hybrid/PHEV,2020,131
4,0810,NT,ICE,2020,21870


In [13]:
# Join other data sources to df

# IF NO NEW DATA RUN THESE LINES
display(ryi[ryi['Postcode']=='0810'])
owners = pd.read_csv('2_staging/vw_owners_clean.csv')


owners['Postcode'] = owners['Postcode'].astype('string').str.zfill(4)
ryi = pd.read_csv('2_staging/id_ryi_clean.csv')
ryi['Postcode'] = ryi['Postcode'].astype('string').str.zfill(4)



data_cube = pd.merge(left=df, right=owners, how='left', on='Postcode')
data_cube = pd.merge(left=data_cube, right=ryi, how='left', on='Postcode')

data_cube = data_cube[['State', 'Postcode', 'Year', 'Fuel Type', 'LVs', 'VWs', 'RYIs']]

data_cube.loc[data_cube['Year'] != 2023, ['VWs','RYIs']] = 0  # set historic years to zero as only have 1 year of data
data_cube.loc[data_cube['Fuel Type'] != 'ICE', 'VWs'] = 0 # only have ICE VWs
data_cube.loc[data_cube['Fuel Type'] != 'BEV', 'RYIs'] = 0 # only have BEV RYIs

# JOIN ON LATITUDE AND LONGITUDE DATA

lat_long = pd.read_csv('1_inputs/geos/postcode_latitude_longitudes.csv')
lat_long['Postcode'] = lat_long['Postcode'].astype('int').astype('string').str.zfill(4)

data_cube = pd.merge(data_cube, lat_long, how='left', on='Postcode')

display(owners[owners['Postcode']=='0810'])

data_cube.to_csv('3_outputs/id_analysis_data_cube.csv')

,Postcode,RYIs
1,0810,4


,Unnamed: 0,Postcode,VWs
9,9,0810,383


In [14]:
data_cube[data_cube['Postcode']=='0810']

,State,Postcode,Year,Fuel Type,LVs,VWs,RYIs,Latitude,Longitude
4,NT,0810,2020,ICE,21870,0.0,0.0,-12.3661,130.8784
5,NT,0810,2020,BEV,6,0.0,0.0,-12.3661,130.8784
6,NT,0810,2020,HFCEV,0,0.0,0.0,-12.3661,130.8784
7,NT,0810,2020,Hybrid/PHEV,288,0.0,0.0,-12.3661,130.8784
10552,NT,0810,2021,ICE,22092,0.0,0.0,-12.3661,130.8784
10553,NT,0810,2021,BEV,16,0.0,0.0,-12.3661,130.8784
10554,NT,0810,2021,HFCEV,0,0.0,0.0,-12.3661,130.8784
10555,NT,0810,2021,Hybrid/PHEV,375,0.0,0.0,-12.3661,130.8784
21100,NT,0810,2022,ICE,21890,0.0,0.0,-12.3661,130.8784
21101,NT,0810,2022,BEV,22,0.0,0.0,-12.3661,130.8784


In [9]:
os.chdir('./1_inputs/demographics/2021_Census_Demographics/census_postal_areas/')


In [10]:
# IMPORT DEMOGRAPHIC DATA

print(os.getcwd())

files = os.listdir()

population = files[0]
pop = pd.read_csv(population, usecols=['POA_CODE_2021','Tot_P_P'])
pop.rename(columns={'POA_CODE_2021':'Postcode', 'Tot_P_P':'Population'}, inplace=True)
pop['Postcode'] = pop['Postcode'].str[3:]
print(pop.sort_values(by='Postcode').head())

os.chdir('../../../../2_staging/')
pop.to_csv('demographic_data.csv')

c:\Users\chris.slim\OneWorkplace\TRACK Australia - General\Clients & Projects\Volkswagen Group Australia\Volkswagen\Data and Tech\ID Series Brief\4. POC Update\Data\1_inputs\demographics\2021_Census_Demographics\census_postal_areas
     Postcode  Population
2579     0800        7149
2580     0810       34330
2581     0812       18634
2582     0820       19920
2583     0822       25304


In [12]:
os.chdir('../')

In [15]:
# REMOVE PII

def strip_pii(loc, field_list):
    """ 
    This function is used to remove PII from a file saved on Sharepoint, without having to open the file or delete it
    loc = location of the file to remove PII from
    field list is a list of fields containing PII that might be present in the dataframe
    """

    z = pd.read_csv(loc)
    
    columns_present = sum([1 if i in z.columns else 0 for i in field_list])

    while columns_present > 0 :
        for field in field_list:
            if field in z.columns:
                z.drop(field, axis=1)
                print('{} removed from CSV'.format(field))
        
            else:
                print('{} was not present in CSV'.format(field))
        
        z.to_csv(loc)
        print('The PII was removed from the dataframe')

        break

    else: print('The dataframe did not contain PII')
    
        

# pii_check = pd.read_csv('1_inputs/ryis/id_ryi_raw_data_update.csv', encoding='iso-8859-1')

# while 'Email' in pii_check.columns:
#     pii_check.drop(['Email','First Name','Last Name'], axis=1).to_csv('1_inputs/ryis/id_ryi_raw_data_update.csv')
#     print('The PII was removed from the ID RYI Data')
#     break

# else: 
#     print('The dataframe did not contain PII')


# x = pd.read_csv('2_staging/ryi_clean_list_check.csv')

# list1 = ['First Name', 'Last Name']
# list2 = x.columns

# sum([1 if i in list2 else 0 for i in list1])

pii_fields = ['Email','First Name', 'Last Name', 'Owner Name: Account Name']

strip_pii('1_inputs/ryis/id_ryi_raw_data_update.csv', pii_fields)
print("\n")
strip_pii('2_staging/ryi_clean_list_check.csv', pii_fields)
print("\n")
strip_pii('1_inputs/vw_owners/vw_owners_raw_data_update.csv', pii_fields)




Email removed from CSV
First Name removed from CSV
Last Name removed from CSV
Owner Name: Account Name was not present in CSV
The PII was removed from the dataframe


Email removed from CSV
First Name removed from CSV
Last Name removed from CSV
Owner Name: Account Name was not present in CSV
The PII was removed from the dataframe




C:\Users\chris.slim\AppData\Local\Temp\ipykernel_10528\491917933.py:10: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  z = pd.read_csv(loc)


Email was not present in CSV
First Name was not present in CSV
Last Name was not present in CSV
Owner Name: Account Name removed from CSV
The PII was removed from the dataframe


: 